In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

def configurar_driver():
    """Configura y retorna el driver de Chrome para Selenium usando webdriver-manager"""
    chrome_options = Options()
    # Opciones para mejorar el rendimiento
    # chrome_options.add_argument("--headless")  # Comentado para poder ver lo que pasa
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    # Usar webdriver_manager para gestionar el chromedriver automáticamente
    servicio = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=servicio, options=chrome_options)
    return driver

def extraer_datos_tabla_desde_archivo_html(ruta_archivo):
    """Extrae los datos de la tabla bursátil desde un archivo HTML local"""
    driver = configurar_driver()
    try:
        # Construir la URL de archivo
        ruta_absoluta = os.path.abspath(ruta_archivo)
        url_archivo = f"file:///{ruta_absoluta}"
        print(f"Abriendo archivo local: {url_archivo}")

        # Abrir el archivo HTML
        driver.get(url_archivo)

        # Esperar a que la tabla se cargue
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )

        # Capturar una captura de pantalla para depuración
        driver.save_screenshot("captura_tabla.png")
        print("Se ha guardado una captura de pantalla como 'captura_tabla.png'")

        # Intentar localizar la tabla
        tablas = driver.find_elements(By.TAG_NAME, "table")
        print(f"Se encontraron {len(tablas)} tablas en la página")

        if not tablas:
            print("No se encontraron tablas. Intentando con otro selector...")
            # Intentar con diferentes selectores
            tablas = driver.find_elements(By.CSS_SELECTOR, ".table, [class*='tabla']")
            print(f"Segundo intento: Se encontraron {len(tablas)} tablas con selector CSS")

        if not tablas:
            print("Aún no se encuentran tablas. Buscando cualquier elemento con filas...")
            filas = driver.find_elements(By.TAG_NAME, "tr")
            print(f"Se encontraron {len(filas)} filas directamente")

            # Si hay filas pero no tablas, procesamos las filas directamente
            if filas:
                return procesar_filas(filas)
            return None

        # Trabajar con la primera tabla encontrada
        tabla = tablas[0]
        filas = tabla.find_elements(By.TAG_NAME, "tr")
        print(f"La tabla seleccionada tiene {len(filas)} filas")

        return procesar_filas(filas)

    except Exception as e:
        print(f"Error al extraer datos desde archivo: {e}")
        return None

    finally:
        driver.quit()

def procesar_filas(filas):
    """Procesa las filas para extraer los datos"""
    try:
        # Extraer encabezados
        encabezados = []
        if filas:
            encabezado_fila = filas[0]
            # Intentar con th primero (encabezados de tabla)
            columnas_encabezado = encabezado_fila.find_elements(By.TAG_NAME, "th")

            # Si no hay th, intentar con td
            if not columnas_encabezado:
                columnas_encabezado = encabezado_fila.find_elements(By.TAG_NAME, "td")

            for columna in columnas_encabezado:
                encabezados.append(columna.text.strip())

            print(f"Encabezados encontrados: {encabezados}")

        # Extraer datos
        datos = []
        for i in range(1, len(filas)):
            fila = filas[i]
            columnas = fila.find_elements(By.TAG_NAME, "td")
            fila_datos = []
            for columna in columnas:
                fila_datos.append(columna.text.strip())

            if fila_datos and any(fila_datos):  # Asegurarse de que la fila tiene datos
                datos.append(fila_datos)

        print(f"Se extrajeron {len(datos)} filas de datos")

        # Crear DataFrame
        if encabezados and datos and len(encabezados) == len(datos[0]):
            df = pd.DataFrame(datos, columns=encabezados)
        else:
            # Si no se pudieron extraer los encabezados correctamente o no coinciden
            print("Los encabezados no coinciden con los datos. Creando DataFrame sin encabezados específicos.")
            df = pd.DataFrame(datos)

        return df

    except Exception as e:
        print(f"Error al procesar filas: {e}")
        return None

def extraer_datos_por_identificadores_especificos():
    """Método para extraer datos buscando contenedores específicos"""
    driver = configurar_driver()
    try:
        # Tomar una captura de pantalla para ver el estado del DOM
        driver.save_screenshot("captura_completa.png")
        print("Se ha guardado una captura de pantalla como 'captura_completa.png'")

        # Basándonos en la imagen, vamos a buscar por los identificadores que se ven en la tabla
        # Extraer todos los elementos que contienen textos como "ALICORP S.A.A.", "BVL", etc.
        elementos_empresa = driver.find_elements(By.XPATH, "//*[contains(text(), 'ALICORP') or contains(text(), 'UNION DE CERVECERIAS') or contains(text(), 'BANCO')]")
        print(f"Se encontraron {len(elementos_empresa)} elementos con nombres de empresas")

        datos = []
        for elemento in elementos_empresa:
            try:
                # Obtener el elemento padre que contiene toda la fila
                fila = elemento.find_element(By.XPATH, "./ancestor::tr")

                # Extraer todas las celdas de la fila
                celdas = fila.find_elements(By.TAG_NAME, "td")
                textos_celdas = [celda.text.strip() for celda in celdas]

                if textos_celdas:
                    datos.append(textos_celdas)
            except Exception as e:
                print(f"Error al procesar elemento: {e}")
                continue

        # Crear DataFrame
        if datos:
            df = pd.DataFrame(datos)
            return df

        return None

    except Exception as e:
        print(f"Error general en extracción específica: {e}")
        return None

    finally:
        driver.quit()

# Función para guardar la página HTML
def guardar_html_pagina(url, nombre_archivo="pagina.html"):
    """Guarda el HTML de la página para análisis posterior"""
    driver = configurar_driver()
    try:
        driver.get(url)
        time.sleep(3)  # Esperar a que cargue

        # Obtener el HTML
        html = driver.page_source

        # Guardar en archivo
        with open(nombre_archivo, "w", encoding="utf-8") as f:
            f.write(html)

        print(f"HTML guardado en '{nombre_archivo}'")

        # Tomar captura de pantalla
        driver.save_screenshot("captura_pagina.png")
        print("Captura de pantalla guardada en 'captura_pagina.png'")

        return True
    except Exception as e:
        print(f"Error al guardar HTML: {e}")
        return False
    finally:
        driver.quit()


In [8]:

# Ejemplo de uso
if __name__ == "__main__":
    # 1. Primero intentamos con un archivo HTML local si existe
    archivo_html = "pagina_bvl.html"
    if os.path.exists(archivo_html):
        print(f"Usando archivo HTML local: {archivo_html}")
        df = extraer_datos_tabla_desde_archivo_html(archivo_html)
        if df is not None and not df.empty:
            print("\nDatos extraídos del archivo HTML local:")
            print(df.head())
            df.to_csv("datos_bursatiles_local.csv", index=False)
            print("Datos guardados en 'datos_bursatiles_local.csv'")
        else:
            print("No se pudieron extraer datos del archivo HTML local.")
    else:
        print(f"No se encontró el archivo HTML local: {archivo_html}")

        # 2. Si no hay archivo HTML, pedir URL al usuario
        url = input("Ingresa la URL de la página con los datos bursátiles (deja en blanco para usar la URL por defecto): ")
        if not url:
            url = "https://bvl.com.pe/mercado/movimientos-diarios"  # URL por defecto

        # Guardar el HTML para análisis posterior
        print(f"\nGuardando HTML de la página {url}...")
        guardado = guardar_html_pagina(url, "pagina_bvl.html")

        if guardado:
            print("\nEl HTML se ha guardado correctamente. Ahora puedes volver a ejecutar este script para analizar el archivo HTML local.")
        else:
            print("\nNo se pudo guardar el HTML. Intenta acceder a la página manualmente y guardar el HTML desde tu navegador.")

No se encontró el archivo HTML local: pagina_bvl.html

Guardando HTML de la página https://bvl.com.pe/mercado/movimientos-diarios...
Error al guardar HTML: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=134.0.6998.88)
Stacktrace:
	GetHandleVerifier [0x00C7B5A3+24387]
	(No symbol) [0x00C05904]
	(No symbol) [0x00AE0753]
	(No symbol) [0x00ADDB9E]
	(No symbol) [0x00AD14B8]
	(No symbol) [0x00AD2DC0]
	(No symbol) [0x00AD1747]
	(No symbol) [0x00AD12A3]
	(No symbol) [0x00AD0FB1]
	(No symbol) [0x00ACEE80]
	(No symbol) [0x00ACF91B]
	(No symbol) [0x00AE420E]
	(No symbol) [0x00B6FDE7]
	(No symbol) [0x00B4D84C]
	(No symbol) [0x00B6F138]
	(No symbol) [0x00B4D646]
	(No symbol) [0x00B1C59F]
	(No symbol) [0x00B1D8E4]
	GetHandleVerifier [0x00F7D883+3179043]
	GetHandleVerifier [0x00F96CF9+3282585]
	GetHandleVerifier [0x00F9167C+3260444]
	GetHandleVerifier [0x00D14330+650448]
	(No symbol) [0x00C0ED0D]
	(No symbol) [0x00C0BAF8]
	(No symbol) [0x00C0BC99]
	(No symbol) [0x00BFE530]